# Интерфейс

Для упрощения создания собственных цепочек в LCEL релизован протокол [Runnable](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable).
Протокол стандартизирует определение и вызов цепочек и поддерживается в большинстве компонентов.
Стандартные методы интерфеса Runnable:

* [`stream`](#stream) — потоковая передача частей ответа;
* [`invoke`](#invoke) — вызов цепочки с входными данными;
* [`batch`](#batch) — вызов цепочки со списком входных данных.

Асинхронные версии стандартных методов:

- [`astream`](#async-stream) — асинхронная поотковая передача частей ответа;
- [`ainvoke`](#async-invoke) — асинхронный вызов цепочки с входными данными;
- [`abatch`](#async-batch) — вызов цепочки со списком входных данных;
- [`astream_log`](#async-stream-intermediate-steps) — наряду с потоковой передачей ответа передает промежуточные шаги по мере их возникновения;
- [`astream_events`](#async-stream-events) — **beta** потоковая передача событий по мере их возникновения в цепочке. Метод доступен начиная с `langchain-core` версии 0.1.14.

Типы входных и выходных данных отличаются в зависимости от компонента:

| Компонент | Тип ввода | Тип вывода |
| --- | --- | --- |
| Prompt | Словарь | PromptValue |
| ChatModel | Одиночная строка, список сообщений чата или PromptValue | ChatMessage |
| LLM | Одиночная строка, список сообщений чата или PromptValue | Строка |
| OutputParser | Вывод LLM или ChatModel | Зависит от парсера |
| Retriever | Одиночная строка | Список документов |
| Tool | Одиночная строка или словарь, в зависимости от инструмента | Зависит от инструмента |


Все экземпляры Runnable предоставляют доступ к схемам входных и выходных данных для проверки ввода и вывода:

* [`input_schema`](#input-schema) — модель Pydantic, сгенерированная на основе структуры Runnable;
* [`output_schema`](#output-schema) — модель Pydantic, сгенерированная на основе структуры Runnable.

Давайте рассмотрим эти методы. Для этого мы создадим очень простой шаблон промпта + модель чата.

Рассмотрим эти методы на примере работы простой цепочки, которая состоит из шаблона промпта и модели (*PromptTemplate + ChatModel*»).

In [ ]:
%pip install --upgrade --quiet  gigachain gigachat

In [3]:
from langchain.chat_models.gigachat import GigaChat
from langchain_core.prompts import ChatPromptTemplate

model = GigaChat(credentials="<атворизационные_данные>", verify_ssl_certs=False)
prompt = ChatPromptTemplate.from_template("расскажи шутку про {topic}")
chain = prompt | model

## Схема входных данных

Описание входных данных, которые принимает экземпляр Runnable.
Представляет собой модель Pydantic, которая динамически генерируется на основе структуры экземпляра Runnable.
Чтобы получить представление данных в формате JSONSchema, вызовите метод `.schema()`.

In [4]:
# Схема входных данных цепочки соответствует схеме ее первой части — промпта.
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [5]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [6]:
model.input_schema.schema()

{'title': 'GigaChatInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'

## Схема выходных данных

Описание выходных данных, которые возвращает экземпляр Runnable.
Представляет собой модель Pydantic, которая динамически генерируется на основе структуры экземпляра Runnable.
Чтобы получить представление данных в формате JSONSchema, вызовите метод `.schema()`.

In [7]:
# Схема выходных данных цепочки — это схема вывода ее последней части. В данном случае это модель, которая возвращает сообщение чата.
chain.output_schema.schema()

{'title': 'GigaChatOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'name': {'title': 'Name', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}},

## Потоковая передача

In [10]:
for s in chain.stream({"topic": "облака"}):
    print(s.content, end="", flush=True)

Видны облака из дыма и пара, 
Идут они мимо, идут туда, 
Где небо с землёю цветут чередой, 
Идут они к югу, идут на восток, 
На север идут и на запад идут, 
И каждый идёт по своей кривой.

## Вызов

In [12]:
chain.invoke({"topic": "облака"})

AIMessage(content='Висит на заборе, колышется… Что это? — Это не облако, это — бельё!', response_metadata={'token_usage': Usage(prompt_tokens=18, completion_tokens=27, total_tokens=45), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'})

## Пакетная обработка

In [13]:
chain.batch([{"topic": "облака"}, {"topic": "тучи"}])

[AIMessage(content='Висит на заборе, колышется ветром,\nРыбак на него мочится.\nВот такое у нас сегодня облако.', response_metadata={'token_usage': Usage(prompt_tokens=18, completion_tokens=35, total_tokens=53), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'}),
 AIMessage(content='Вижу, тучи собрались, значит, скоро будет дождь.', response_metadata={'token_usage': Usage(prompt_tokens=18, completion_tokens=16, total_tokens=34), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'})]

Используйте параметр `max_concurrency`, чтобы задать количество одновременных запросов.

In [14]:
chain.batch([{"topic": "облака"}, {"topic": "тучи"}], config={"max_concurrency": 5})

[AIMessage(content='Висит на заборе, колышется ветром,\nРыбак на него мочится.\nВот такое у нас сегодня облако.', response_metadata={'token_usage': Usage(prompt_tokens=18, completion_tokens=35, total_tokens=53), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'}),
 AIMessage(content='Вижу, вижу, собираются тучи, \nТолько гром не грянул бы!\nНадо выпить пропускную рюмку,\nЧтоб её перебить!', response_metadata={'token_usage': Usage(prompt_tokens=18, completion_tokens=41, total_tokens=59), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'})]

## Асинхронная потоковая передача

In [15]:
async for s in chain.astream({"topic": "облака"}):
    print(s.content, end="", flush=True)

Облако с ложкой стоит над тарелкой супа и приговаривает: «Облака-облака, а я вот ем!»

## Асинхронный вызов

In [16]:
await chain.ainvoke({"topic": "облака"})

AIMessage(content='Висит на заборе, колышется… Что это? Это не облако, это мысль.', response_metadata={'token_usage': Usage(prompt_tokens=18, completion_tokens=23, total_tokens=41), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'})

## Асинхронная пакетная обработка

In [17]:
await chain.abatch([{"topic": "облака"}])

[AIMessage(content='Облако с ложкой стоит над тарелкой супа и приговаривает: «Облака-облака, а я вот ем!»', response_metadata={'token_usage': Usage(prompt_tokens=18, completion_tokens=35, total_tokens=53), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'})]

## Асинхронная потоковая передача событий (beta)

Потоковая передача событий находится на стадии бета-версии.

Для корректной работы с API `astream_events`:

* Используйте `async` по всему коду (включая асинхронные инструменты и т. д.).
* При определении собственных функций и Runnable-интерфейсов передавайте обратные вызовы.
* При использовании Runnable-интерфейсов без LCEL, убедитесь, что при работе с LLM вы вызываете `.astream()`, а не `.ainvoke`, чтобы принудительно включать потоковую пердачу токенов.

### Справка по событиям

В таблице представлены некоторые события, которые могут сгенерировать различные Runnable-объекты.
Примеры некоторых их Runnable-объектов приведены после таблицы.

При потоковой передаче входных данных для Runnable-объекта, данные не будут доступны до завершения всего потока. Такое поведение приводит к тому, что входные данные доступны на соответствующем событии `end`, а не на событии `start`.


| Событие                  | Имя             | Фрагмент                        | Ввод                                         | Вывод                                          |
|--------------------------|-----------------|---------------------------------|----------------------------------------------|------------------------------------------------|
| on_chat_model_start      | [имя модели]    |                                 | {"messages": [[SystemMessage, HumanMessage]]} |                                                |
| on_chat_model_stream     | [имя модели]    | AIMessageChunk(content="hello") |                                              |                                                |
| on_chat_model_end        | [имя модели]    |                                 | {"messages": [[SystemMessage, HumanMessage]]} | {"generations": [...], "llm_output": None, ...}   |
| on_llm_start             | [имя модели]    |                                 | {'input': 'hello'}                           |                                                |
| on_llm_stream            | [имя модели]    | 'Привет'                        |                                              |                                                |
| on_llm_end               | [имя модели]    |                                 | 'Hello human!'                           |
| on_chain_start           | format_docs     |                                 |                                              |                                                |
| on_chain_stream          | format_docs     | "привет мир!, прощай мир!"      |                                              |                                                |
| on_chain_end             | format_docs     |                                 | [Document(...)]                              | "hello world!, goodbye world!"                 |
| on_tool_start            | some_tool       |                                 | {"x": 1, "y": "2"}                           |                                                |
| on_tool_stream           | some_tool       | {"x": 1, "y": "2"}              |                                              |                                                |
| on_tool_end              | some_tool       |                                 |                                              | {"x": 1, "y": "2"}                             |
| on_retriever_start       | [имя ретривера] |                                 | {"query": "hello"}                           |                                                |
| on_retriever_chunk       | [имя ретривера] | {документы: [...]}              |                                              |                                                |
| on_retriever_end         | [имя ретривера] |                                 | {"query": "hello"}                           | {documents: [...]}                             |
| on_prompt_start          | [имя шаблона]   |                                 | {"question": "hello"}                         |                                                |
| on_prompt_end            | [имя шаблона]   |                                 | {"question": "hello"}                         | ChatPromptValue(сообщения: [SystemMessage, ...])|

Примеры объектов, связанные с событиями, представленными в таблице:

`format_docs`:

```python
def format_docs(docs: List[Document]) -> str:
    '''Format the docs.'''
    return ", ".join([doc.page_content for doc in docs])

format_docs = RunnableLambda(format_docs)
```

`some_tool`:

```python
@tool
def some_tool(x: int, y: str) -> dict:
    '''Some_tool.'''
    return {"x": x, "y": y}
```

`prompt`:

```python
template = ChatPromptTemplate.from_messages(
    [("system", "You are Cat Agent 007"), ("human", "{question}")]
).with_config({"run_name": "my_template", "tags": ["my_template"]})
```



Определим новую цепочку, чтобы более наглядно показать работу интерфейса `astream_events`, а затем и `astream_log`.

In [18]:
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Отвечай на вопрос только на основе контекста:
{context}

Вопрос: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["Василий работал в Сбербанке"],
    embedding=GigaChatEmbeddings(
        credentials="<авторизационные_данные>", verify_ssl_certs=False
    ),
)
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model.with_config(run_name="my_llm")
    | StrOutputParser()
)

Используем `astream_events`, чтобы получить события от ретривера и LLM.

In [ ]:
async for event in retrieval_chain.astream_events(
    "где работал Василий?", version="v1", include_names=["Docs", "my_llm"]
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="|")
    elif kind in {"on_chat_model_start"}:
        print()
        print("Потоковая передача от LLM:")
    elif kind in {"on_chat_model_end"}:
        print()
        print("Завершение потоковой передачи от LLM.")
    elif kind == "on_retriever_end":
        print("--")
        print("Полученные документы:")
        print(event["data"]["output"]["documents"])
    elif kind == "on_tool_end":
        print(f"Ended tool: {event['name']}")
    else:
        pass

## Асинхронная потоковая передача промежуточных шагов

Во всех Runnable-объектах реализован метод `.astream_log()`, который используется для потоковой передачи всех или части промежуточных шагов вашей цепочки по мере их возникновения.

Метод можно использовать для отображения процесса выполнения пользователю, обработки промежуточных результатов или для отладки цепочки.

Вы можете использовать потоковую передачу всех шагов (по умолчанию) или фильтровать шаги по имени, тегам или метаданным.

Метод возвращает [операции JSONPatch](https://jsonpatch.com), которые при применении в том же порядке, в котором они поступили, позволяют собрать RunState.

```python
class LogEntry(TypedDict):
    id: str
    """ID of the sub-run."""
    name: str
    """Name of the object being run."""
    type: str
    """Type of the object being run, eg. prompt, chain, llm, etc."""
    tags: List[str]
    """List of tags for the run."""
    metadata: Dict[str, Any]
    """Key-value pairs of metadata for the run."""
    start_time: str
    """ISO-8601 timestamp of when the run started."""

    streamed_output_str: List[str]
    """List of LLM tokens streamed by this run, if applicable."""
    final_output: Optional[Any]
    """Final output of this run.
    Only available after the run has finished successfully."""
    end_time: Optional[str]
    """ISO-8601 timestamp of when the run ended.
    Only available after the run has finished."""


class RunState(TypedDict):
    id: str
    """ID of the run."""
    streamed_output: List[Any]
    """List of output chunks streamed by Runnable.stream()"""
    final_output: Optional[Any]
    """Final output of the run, usually the result of aggregating (`+`) streamed_output.
    Only available after the run has finished successfully."""

    logs: Dict[str, LogEntry]
    """Map of run names to sub-runs. If filters were supplied, this list will
    contain only the runs that matched the filters."""
```

### Потоковая передача фрагментов JSONPatch

Вы можете использовать потоковую передачю `JSONPatch` через HTTP-сервер, после чего применяя операции на клиенте восстанавливать RunState.
Используйте [GigaServe](https://github.com/ai-forever/gigaserve) для создания веб-сервера на основе любого Runnable-объекта.

In [21]:
async for chunk in retrieval_chain.astream_log(
    "где работал Василий?", include_names=["Docs"]
):
    print("-" * 40)
    print(chunk)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': 'd8997bd2-c0c5-4818-ac75-7001754a656a',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': 'c780af42-a72e-472a-bef6-cac749136303',
            'metadata': {},
            'name': 'Docs',
            'start_time': '2024-03-28T12:09:01.497+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['map:key:context', 'FAISS', 'GigaChatEmbeddings'],
            'type': 'retriever'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs/final_output',
  'value': {'documents': [Document(page_content='Василий работал в Сбербанке')]}

### Потоковая передача инкрементов RunState

Для получения инкрементов `RunState` достаточно передать аргумент `diff=False`.
Чем больше будет повторяющихся частей, тем более объемным будет вывод.

In [22]:
async for chunk in retrieval_chain.astream_log(
    "где работал Василий?", include_names=["Docs"], diff=False
):
    print("-" * 70)
    print(chunk)

----------------------------------------------------------------------
RunLog({'final_output': None,
 'id': '730fdb7b-e372-4e1b-a72a-d46179877ef1',
 'logs': {},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'})
----------------------------------------------------------------------
RunLog({'final_output': None,
 'id': '730fdb7b-e372-4e1b-a72a-d46179877ef1',
 'logs': {'Docs': {'end_time': None,
                   'final_output': None,
                   'id': 'bd46efbf-b646-4a7b-914a-21222832bc9d',
                   'metadata': {},
                   'name': 'Docs',
                   'start_time': '2024-03-28T12:09:19.793+00:00',
                   'streamed_output': [],
                   'streamed_output_str': [],
                   'tags': ['map:key:context', 'FAISS', 'GigaChatEmbeddings'],
                   'type': 'retriever'}},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'})
-----------------------------------------------------------

## Параллелизм

Рассмотрим поддержку параллелизма в LCEL.
Так, при использовании экземпляра `RunnableParallel`, который обычно записывается в формате словаря, каждый элемент выполняется праллельно.

In [24]:
from langchain_core.runnables import RunnableParallel

chain1 = ChatPromptTemplate.from_template("расскажи шутку про {topic}") | model
chain2 = (
    ChatPromptTemplate.from_template("напиши короткий стих на 2 строчки про {topic}")
    | model
)
combined = RunnableParallel(joke=chain1, poem=chain2)

In [25]:
%%time
chain1.invoke({"topic": "облако"})

CPU times: user 8.92 ms, sys: 2.76 ms, total: 11.7 ms
Wall time: 1.44 s


AIMessage(content='Вот какая шутка у меня получилась: \n\n— Тучка, тучка, а мне не до дождя!..', response_metadata={'token_usage': Usage(prompt_tokens=18, completion_tokens=29, total_tokens=47), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'})

In [27]:
%%time
chain2.invoke({"topic": "облако"})

CPU times: user 8.52 ms, sys: 2.48 ms, total: 11 ms
Wall time: 1.47 s


AIMessage(content='Вот такое стихотворение у меня получилось:\n\nПлывёт облако по небу,\nА я лежать бы мог тут целый век.', response_metadata={'token_usage': Usage(prompt_tokens=24, completion_tokens=32, total_tokens=56), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'})

In [28]:
%%time
combined.invoke({"topic": "облако"})

CPU times: user 28.2 ms, sys: 5.06 ms, total: 33.3 ms
Wall time: 4.58 s


{'joke': AIMessage(content='Вот какая шутка у меня получилась: \n\n— Тучка, тучка, а почему у тебя такие большие ноги?\n\n— А это чтобы по воде ходить!\n\n— А почему у тебя такие большие руки?\n\n— Это чтобы людей обнимать!\n\n— А почему у тебя такой большой нос?\n\n— А это чтобы в темноте с другими тучами не столкнуться!\n\n— А почему у тебя такое лицо бессмысленное?\n\n— Ну, это я ещё не развернулась…', response_metadata={'token_usage': Usage(prompt_tokens=18, completion_tokens=115, total_tokens=133), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'}),
 'poem': AIMessage(content='Вот такое стихотворение у меня получилось:\n\nПлывёт облако по небу,\nА я лежать бы мог тут целый век.', response_metadata={'token_usage': Usage(prompt_tokens=24, completion_tokens=32, total_tokens=56), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'})}

### Параллелизм при пакетной обработке

Параллелизм можно использовать при работе с различными экземплярами Runnsble.
Например, при пакетной обработке.

In [30]:
%%time
chain1.batch([{"topic": "облака"}, {"topic": "тучи"}])

CPU times: user 22.4 ms, sys: 5.31 ms, total: 27.7 ms
Wall time: 1.46 s


[AIMessage(content='Облако на облаке, белый барашек,\nУкололся шипом — и завял цветочек!', response_metadata={'token_usage': Usage(prompt_tokens=18, completion_tokens=26, total_tokens=44), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'}),
 AIMessage(content='Вижу, тучи собрались, сейчас что-то будет… Либо салют, либо дождь.', response_metadata={'token_usage': Usage(prompt_tokens=18, completion_tokens=24, total_tokens=42), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'})]

In [31]:
%%time
chain2.batch([{"topic": "облака"}, {"topic": "тучи"}])

CPU times: user 17.6 ms, sys: 4.51 ms, total: 22.1 ms
Wall time: 1.58 s


[AIMessage(content='Облака плывут по небу,\nОблака летят на юг.', response_metadata={'token_usage': Usage(prompt_tokens=24, completion_tokens=20, total_tokens=44), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'}),
 AIMessage(content='Тучи, как люди, порой собираются вместе,\nТучи, как люди, порой плачут, кричат и смеются.', response_metadata={'token_usage': Usage(prompt_tokens=24, completion_tokens=31, total_tokens=55), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'})]

In [32]:
%%time
combined.batch([{"topic": "облака"}, {"topic": "тучи"}])

CPU times: user 47.2 ms, sys: 10.1 ms, total: 57.3 ms
Wall time: 2.49 s


[{'joke': AIMessage(content='Висит на заборе, колышется ветром,\nРыбак из деревни глядит на неё весь день:\n«Скорей бы выросла!» — думает с надеждой.\nИ так тридцать лет… Ну и что тут ещё сказать?', response_metadata={'token_usage': Usage(prompt_tokens=18, completion_tokens=60, total_tokens=78), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'}),
  'poem': AIMessage(content='Облака плывут по небу,\nОблака, куда же вы?', response_metadata={'token_usage': Usage(prompt_tokens=24, completion_tokens=19, total_tokens=43), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'})},
 {'joke': AIMessage(content='Вижу, тучи собираются, значит, будет дождик.', response_metadata={'token_usage': Usage(prompt_tokens=18, completion_tokens=16, total_tokens=34), 'model_name': 'GigaChat-Pro-preview:2.2.25.3', 'finish_reason': 'stop'}),
  'poem': AIMessage(content='Тучи, словно камни, висят над головой,\nТучи не дают нам увидеть небо голубое.', response_metadata={'tok